# Week 2 - Objectives

1. Students will define classes and default, copy, and general constructors.
2. Students will write code that calls each one of the three types of constructors above.

## Reading:

[C++ Primer](https://cpp-primer.pages.dev/), Chapter 7:

1. Section 7.5 Constructors revisited (Attn!, C++11 rules)

[cppreference.com - constructors and member initializer lists](https://www.en.cppreference.com/w/cpp/language/initializer_list.html) - mentions the differences between the C++ standards.

In [1]:
// setup
#include <iostream>

# OOP - initialization: constructors
Since data members should be private, we need a **function** to initialize data members. This function is called a constructor. The following kinds of constructors are defined, depending on the number of arguments:
  - no arguments = default constructor
  - one argument which is an object of the same class = copy constructor
  - one or more arguments, not of the same class = general constructor
  
The constructor syntax is special, unlike that of declaring member functions.

In [ ]:
class Rational {
    int num, den;
    
  public:
    // default
    Rational() {
        std::cout << "Rational()" << std::endl;
    }
    
    // copy
    Rational(const Rational &r) {
        std::cout << "Rational(const Rational&)" << std::endl;
    }
    
    // general constructor
    Rational(int n, int d) {
        std::cout << "Rational(int, int)" << std::endl;
    }
}

**Syntax for defining constructors:**
  - one
  - two

In [ ]:
// which constructors are called?
Rational r1;

In [ ]:
Rational r2 = r1;

In [ ]:
Rational r3(1,2);

In [ ]:
Rational r4 = {2,3};

In [ ]:
Rational r5 = Rational(10,20);

In [ ]:
void dummy(Rational r) {
    std::cout << "dummy(Rational)" << std::endl;
}

void dummy_ref(Rational & r) {
    std::cout << "dummy_ref(Rational &)" << std::endl;
}

In [ ]:
dummy(r1);

In [ ]:
dummy_ref(r1);

In [ ]:
r1 = r2;

**Syntax for calling constructors**

1. One

**Exercises**

1. Remove the debug/print statements from the rational class constructors and add useful initialization code. Remember that a rational cannot have a zero denominator.

2. Think about how to handle the situation when a programmer declares a rational with a zero denominator:
`Rational r(1,0);`.

# The Yin and Yang of object initialization - destructors

When an object goes out of scope (dies), a special function called **destructor** is called. Goal: to free any resources that the object has acquired.

In [2]:
class Rational {
    int num, den;
    
  public:
    // default
    Rational() {
        std::cout << "Rational()" << std::endl;
    }
    
    // copy
    Rational(const Rational &r) {
        std::cout << "Rational(const Rational&)" << std::endl;
    }
    
    // general constructor
    Rational(int n, int d) {
        std::cout << "Rational(int, int)" << std::endl;
    }
    
    // destructor
    ~Rational() {
        std::cout << "~Rational()" << std::endl;
    }
}

**Syntax for defining destructors:**
1. One 

In [4]:
// a destructor is called when an object goes out of scope.
// So, we use a block statement to force this
{
    Rational r1;
    std::cout << "Do some work and we are done." << std::endl;
}

Rational()
Do some work and we are done.
~Rational()


**Syntax for calling destructors:**
1. Destructors are not called explicitly by the programmer.

# OOP - Resource Acquisition Is Initialization (RAII) #

Rule: create an object whenever your program must acquire a resource (memory, socket, file, mutex, etc.) 

  - acquire the resource in the constructor
  - release the resource in the destructor

Example: read the first character from each file in the current directory and print the character code on console.